# Get repsonse from an LLM


To get started, [get an API key](https://g.co/ai/idxGetGeminiKey) and replace the word `API KEY` below with your API key:

Build your own Agent 

In [ ]:
import google.generativeai as genai
from dotenv import load_dotenv
import os
import re

# Load environment variables from .env file
load_dotenv()

# Access the API key from the environment variable
api_key = os.getenv("SECRET_KEY")

# Configure the model
genai.configure(api_key=api_key)
model = genai.GenerativeModel("gemini-2.0-flash")

# Define the prompt
base_prompt = """
You cycle through Thought, Action, PAUSE, Observation. At the end of the loop you output a final Answer. Your final answer should be highly specific to the observations you have from running
the actions.
1. Thought: Describe your thoughts about the question you have been asked.
2. Action: run one of the actions available to you - then return PAUSE.
3. PAUSE
4. Observation: will be the result of running those actions.

Available actions:
- getCurrentWeather: 
    E.g. getCurrentWeather: Salt Lake City
    Returns the current weather of the location specified.
- getLocation:
    E.g. getLocation: null
    Returns user's location details. No arguments needed.

Example session:
Question: Please give me some ideas for activities to do this afternoon.
Thought: I should look up the user's location so I can give location-specific activity ideas.
Action: getLocation: null
PAUSE

You will be called again with something like this:
Observation: "New York City, NY"

Then you loop again:
Thought: To get even more specific activity ideas, I should get the current weather at the user's location.
Action: getCurrentWeather: New York City
PAUSE

You'll then output:
Answer: <Suggested activities based on sunny weather that are highly specific to New York City and surrounding areas.>
"""

# Dummy functions
def recallConvo(place=None):
    # Debug
    return {}

def addToRecall(place=None):
    # Debug
    return {}

available_functions = {
    "recallConvo": recallConvo,
    "addToRecall": addToRecall
}

# Regex for parsing actions
action_regex = r"^Action: (\w+): (.*)$"

# Initialize variables
observation = None
max_calls = 5

# Start the loop
for i in range(max_calls):
    print(f"\n--- Call {i + 1} ---")
    
    # Build the dynamic prompt
    if observation:
        full_prompt = f"{base_prompt}\n\nUser: {query}\nAssistant: {observation}\n\nUser:"
    else:
        full_prompt = f"{base_prompt}\n\nUser: {query}\nAssistant:"

    # Generate response from the model
    response = model.generate_content(full_prompt)
    response_text = response.parts[0].text.strip()
    print(f"Response:\n{response_text}")

    # Split response text into lines
    response_lines = response_text.split("\n")
    
    # Find the action string in the response
    found_action_str = next((line for line in response_lines if re.match(action_regex, line)), None)
   

    if found_action_str:
        # Parse the action and arguments
        actions = re.match(action_regex, found_action_str)
        if actions:
            action, action_arg = actions.groups()
            if action in available_functions:
                # Execute the action
                observation = available_functions[action](action_arg)
            else:
                print(f"Invalid action: {action}")
                break
        else:
            print("No valid action match.")
            break
    else:
        print("No valid action found in response.")
        break


: 

On the second iteration  the model provides repsponse based on the user's location. The code terminates the call to the model  as it has already found the user's location. 